In [1]:


import os, json
import torch



In [2]:


csc_train_path = "csc/train.jsonl"
csc_test_path = "csc/test.jsonl"
csc_val_path = "csc/validation.jsonl"


with open(csc_train_path, "r", encoding="utf-8") as fin:
    for line in fin:
        print(line)
        break




{"conversations":[{"from":"human","value":"对这个句子语法纠错\n\n别只能想自己，想你周围的人。还有你，如果你是一个家庭的爸爸，你多想自己的孩子；如果你是青少年你多想自己的未来；那你可以禁烟了。"},{"from":"gpt","value":"别只想自己，要想想你周围的人。还有，如果你是一个家庭的爸爸，你要多想想自己的孩子；如果你是青少年你要多想想自己的未来；那你就可以戒烟了。"}]}



In [3]:
from modelscope.msdatasets import MsDataset

dataset = MsDataset.load("csc/")


In [4]:
dataset

{'train': <modelscope.msdatasets.ms_dataset.MsDataset at 0x7eff547e79a0>,
 'test': <modelscope.msdatasets.ms_dataset.MsDataset at 0x7eff547e7f70>,
 'validation': <modelscope.msdatasets.ms_dataset.MsDataset at 0x7eff547e7a30>}

In [5]:


dataset["train"][0]

{'conversations': [{'from': 'human',
   'value': '对这个句子语法纠错\n\n别只能想自己，想你周围的人。还有你，如果你是一个家庭的爸爸，你多想自己的孩子；如果你是青少年你多想自己的未来；那你可以禁烟了。'},
  {'from': 'gpt',
   'value': '别只想自己，要想想你周围的人。还有，如果你是一个家庭的爸爸，你要多想想自己的孩子；如果你是青少年你要多想想自己的未来；那你就可以戒烟了。'}]}

In [6]:

dataset["train"][1]



{'conversations': [{'from': 'human',
   'value': '对这个句子语法纠错\n\n若父母主动和孩子平起平坐，倾心交谈，互相充分理解对方，会能够大幅减少互相抱怨的情况。'},
  {'from': 'gpt', 'value': '若父母主动和孩子平起平坐，倾心交谈，互相充分理解对方，会大幅减少互相抱怨的情况。'}]}

In [7]:

"""
对数据进行转换成swift需要的数据集
"""

from tqdm import tqdm

def judge_sentence_correct(example):
    system_prompt = "你是一个语法纠错专家，文本中的错误可能包含错别字、漏字、多字，你可以通过增删改查的方式进行文本语法纠错。"
    conversations = example["conversations"]
    human_message = conversations[0]["value"]
    response = conversations[1]["value"]

    instruction, inputs = human_message.split("\n\n")

    # response_json = json.dumps({"correct": inputs==response, "correct_text": response}, ensure_ascii=False)

    prompt_example = {"system": system_prompt, "query": inputs, "response": response}
    # prompt_example = {"system": system_prompt, "query": inputs, "response": response_json}

    # if inputs == response:
    #     return False, inputs, response
    # else:
    #     return True, inputs, response
    return prompt_example


# print(judge_sentence_correct(dataset["train"][0]))
    

def gen_sft_prompt_json(json_path):
    all_sft_prompt_example_list = []
    
    with open(json_path, "r", encoding="utf-8") as fin:
        for line in fin:
            prompt_example = judge_sentence_correct(line.strip())
            all_sft_prompt_example_list.append(prompt_example)
    return all_sft_prompt_example_list


def gen_sft_prompt_json_from_dir(json_dir):

    dataset = MsDataset.load(json_dir)

    all_sft_prompt_example_list = []
    for example in tqdm(dataset["train"]):
        prompt_example = judge_sentence_correct(example)
        all_sft_prompt_example_list.append(prompt_example)
    return all_sft_prompt_example_list



all_sft_prompt_example_list = gen_sft_prompt_json_from_dir("csc/")
num_valid_example = 0
with open("train_sft.jsonl", "w", encoding="utf-8") as fout:
    for prompt_example in all_sft_prompt_example_list:
        if len(prompt_example["system"] + prompt_example["query"] + prompt_example["response"]) < 100:   # 过滤，避免过长
            fout.write(json.dumps(prompt_example, ensure_ascii=False) + "\n")
            num_valid_example += 1

print("微调样本数：", num_valid_example)




100%|██████████| 1201765/1201765 [00:23<00:00, 51807.27it/s]


微调样本数： 692206


In [8]:
print(len(all_sft_prompt_example_list))

1201765


In [9]:
all_sft_prompt_example_list[:2]

max_len_seq = 0

seq_lens = []

for example in all_sft_prompt_example_list:
    seq_lens.append(len(example["system"] + example["query"] + example["response"]))
    max_len_seq = max(max_len_seq, len(example["system"] + example["query"] + example["response"]))

print(max_len_seq)


889


In [10]:
from collections import Counter

cc = Counter(seq_lens)

cc


Counter({80: 30395,
         78: 30207,
         76: 30159,
         74: 29803,
         82: 28864,
         84: 28505,
         72: 28504,
         86: 27078,
         88: 25883,
         90: 24561,
         70: 24167,
         92: 23765,
         94: 22742,
         96: 21198,
         68: 20580,
         98: 19473,
         100: 17961,
         102: 17319,
         66: 16407,
         77: 16294,
         81: 15969,
         79: 15956,
         83: 15621,
         75: 15586,
         104: 15492,
         85: 15029,
         106: 14718,
         87: 14632,
         73: 14586,
         89: 13883,
         108: 13863,
         110: 13277,
         91: 13006,
         71: 12984,
         112: 12701,
         93: 12234,
         114: 11944,
         95: 11500,
         64: 11475,
         116: 11412,
         69: 10979,
         118: 10929,
         97: 10740,
         120: 10655,
         99: 10119,
         122: 10034,
         124: 9654,
         126: 9296,
         101: 9071,
        